# Preprocessing pipeline tutorial

## Outline

<img src="static/preprocessing_pipeline_diagram.svg">

1. __Temporal filtering__

High-frequency artefacts and slow drifts are removed with a zero-phase bandpass filter 
using mne-Python [1]. 

2. __Segmenting the data__

Epochs are non-overlapping data segments created from the continuous data with a 
given duration.
Epochs can be created from (1) events; there is a custom method that created epochs 
based on annotations in the raw data, (2) without events, data segments are created 
from the beginning of the raw data. 

3. __Outlier data rejection__  

- _Preliminar rejection_

Epochs are rejected based on a global threshold on the z-score (> 3) of the epoch 
variance and amplitude range.

- _ICA decomposition_  

The default method is the infomax algorithm, however it can be changed in the 
configuration file along with the number of components and the decimation parameter. 
Components containing blink artefacts are automatically marked with mne-Python.
The ICA sourced can be visualized and interactively selected and rejected based on 
their topographies, time-courses or frequency spectra.

- _Autoreject_  

Autoreject [2, 3] uses unsupervised learning to estimate the rejection threshold for 
the epochs. In order to reduce computation time that increases with the number of 
segments and channels, autoreject can be fitted on a representative subset of epochs 
(25% of total epochs). Once the parameters are learned, the solution can be applied to 
any data that contains channels that were used during fit.

4. __Outlier channel interpolation__

The Random Sample Consensus (RANSAC) algorithm [4] selects a random subsample of good 
channels to make predictions of each channel in small non-overlapping 4 seconds long 
time windows. It uses a method of spherical splines (Perrin et al., 1989) to 
interpolate the bad sensors.


#### References

[1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, ISSN 1662-453X

[2] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.” In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[3] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”. NeuroImage, 159, 417-429.

[4] Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., & Robbins, K. A. (2015). The PREP pipeline: standardized preprocessing for large-scale EEG analysis. Frontiers in neuroinformatics, 9, 16.



## Import packages


```%matplotlib qt``` is the recommended backend for interactive visualization (can be slower);    

switch to ```%matplotlib inline``` for faster but static plots

In [3]:
import os
from pathlib import Path

from ipyfilechooser import FileChooser
import pandas as pd

from meeg_tools.preprocessing import *
from meeg_tools.utils.epochs import create_epochs_from_events, create_metadata
from meeg_tools.utils.raw import read_raw_measurement, filter_raw
from meeg_tools.utils.log import update_log

%matplotlib qt

## Load raw data


See [this](https://mne.tools/stable/auto_tutorials/io/20_reading_eeg_data.html) documentation for help with supported file formats.  


In [7]:
# Use the widget to navigate to the experiment folder path and select an EEG file 
base_path = '/Users/weian/Library/Mobile Documents/com~apple~CloudDocs/crnl/eeg-workshop/data'
fc = FileChooser(base_path)
fc.filter_pattern = ['*.vhdr', '*.edf']

display(fc)

FileChooser(path='/Users/weian/Library/Mobile Documents/com~apple~CloudDocs/crnl/eeg-workshop/data', filename=…

In [8]:
fc.selected

'/Users/weian/Library/Mobile Documents/com~apple~CloudDocs/crnl/eeg-workshop/data/61_E/Day1/EEG/61_E_Day1.vhdr'

In [9]:
# Load selected file
raw = read_raw_measurement(raw_file_path=fc.selected)

Extracting parameters from /Users/weian/Library/Mobile Documents/com~apple~CloudDocs/crnl/eeg-workshop/data/61_E/Day1/EEG/61_E_Day1.vhdr...
Setting channel info structure...


In [10]:
print(raw.info)

<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 64 items (64 EEG)
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2021-02-08 14:08:06 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 temp: 61_E_Day1
>


In [ ]:
#raw.copy().crop(tmin=600, tmax=1200).plot()

## Select condition

The current logic for saving the preprocessed files is to create subfolders inside `base_path`,
with the name "preprocessed" and the name of the condition (e.g. "epochs_asrt", "epochs_rs").

In [11]:
condition = 'epochs_asrt'


# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
epochs_path = os.path.join(base_path, folder_name, condition)


# Create path to epoch files
if not os.path.exists(epochs_path):
    os.makedirs(epochs_path)
    
print(epochs_path)

/Users/weian/Library/Mobile Documents/com~apple~CloudDocs/crnl/eeg-workshop/data/preprocessed/epochs_asrt


## Temporal filtering

We apply a bandpass filter on the continuous data using the `filter_raw` function.

The default parameters can be checked with `settings['bandpass_filter']`

In [12]:
settings["bandpass_filter"]

{'low_freq': 0.5, 'high_freq': 45}

In [13]:
raw_bandpass = filter_raw(raw=raw)

Reading 0 ... 3250369  =      0.000 ...  6500.738 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 3301 samples (6.602 sec)



[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    6.3s
[Parallel(n_jobs=8)]: Done  64 out of  64 | elapsed:   16.8s finished


## Create epochs

### Create epochs for event-related analysis

We create epochs from __selected__ events (stimuli) in the data.

Epochs are created with respect to the stimulus onset defined by `start_time` and 
`end_time` within `settings['epochs']`.

In [14]:
settings['epochs']

{'start_time': 0.0, 'end_time': 1.0, 'duration': 1}

In [ ]:
settings

In [15]:
settings['epochs']['start_time'] = -0.250
settings['epochs']['end_time'] = 1.2

In [16]:
events_ids = np.concatenate([np.arange(10, 53, 1), 
                             np.arange(10, 53, 1) + 100,
                            [211, 212, 213, 214, 215, 216]]) # boundaries of epochs

print(events_ids)

[ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 146 147 148 149 150 151 152 211 212 213 214
 215 216]


In [17]:
epochs = create_epochs_from_events(raw=raw_bandpass, event_ids=events_ids)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 24', 'Stimulus/S 25', 'Stimulus/S 26', 'Stimulus/S 27', 'Stimulus/S 28', 'Stimulus/S 29', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 32', 'Stimulus/S 34', 'Stimulus/S 35', 'Stimulus/S 36', 'Stimulus/S 37', 'Stimulus/S 38', 'Stimulus/S 39', 'Stimulus/S 40', 'Stimulus/S 41', 'Stimulus/S 42', 'Stimulus/S 43', 'Stimulus/S 44', 'Stimulus/S 45', 'Stimulus/S 46', 'Stimulus/S 47', 'Stimulus/S 48', 'Stimulus/S 49', 'Stimulus/S 51', 'Stimulus/S 52', 'Stimulus/S 61', 'Stimulus/S 62', 'Stimulus/S 63', 'Stimulus/S 64', 'Stimulus/S 65', 'Stimulus/S 66', 'Stimulus/S 67', 'Stimulus/S 68', 'Stimulus/S 69', 'Stimulus/S 70', 'Stimulus/S 71', 'Stimulus/S 72', 'Stimulus/S 75', 'Stimulus/S 76', 'Stimulus/S 77', 'Stimulus/S 78', 

In [ ]:
epochs['10'].average().plot()

## Create metadata for epochs (optional)

- adding metadata makes it easier to select epochs of different types
- custom triggers are selected from the raw instance

- metadata can be added or replaced later (e.g. after preprocessing)

In [18]:
metadata = create_metadata(epochs=epochs)

# We have to assign it to the epochs instance to take effect
epochs.metadata = metadata

Found these indices for these epoch boundary events: 
211	1345
212	2687
213	4018
214	5351
215	6678
Adding metadata with 8 columns


In [ ]:
epochs.metadata

In [ ]:
#epochs["triplet == 'L' | triplet == 'H'"].metadata

In [21]:
# subselecting epochs 
# Here we could also include thrills, repetitions, or practice stimuli.
# ICA should not run on duplicate data (epochs should not be overlapping!)

epochs = epochs["triplet == 'L' | triplet == 'H'"]
epochs = epochs["answer == 'correct'"]

In [ ]:
#epochs.metadata.head()

In [ ]:
epochs[::7].plot()

## Run preprocessing


### 1.1. Preliminary epoch rejection

In [22]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

Preliminary epoch rejection: 
Loading data for 1599 events and 726 original time points ...
0 bad epochs dropped
Bad epochs by amplitude
	[   9   27   32   49   50  110  130  137  139  191  268  269  295  319
  498  753  754  761  815  905  906 1166 1221 1344 1345 1417 1481 1522]
Bad epochs by deviation
	[  10   12   27   28   32   33   47   49   50   61   62  110  130  138
  139  163  182  191  192  267  268  269  270  294  295  296  310  319
  320  325  335  336  337  408  497  498  519  523  525  561  562  578
  600  622  623  752  753  760  761  814  905  906  926  927  968  970
 1013 1014 1158 1166 1168 1220 1344 1345 1346 1358 1418 1420 1429 1430
 1472 1551]
Bad epochs by variance
	[   9   24   27   32   49   50   66  110  130  137  139  191  235  268
  269  295  319  327  335  408  411  498  753  754  761  815  905  906
  926 1166 1221 1344 1345 1417 1472 1478 1480 1481 1484 1488 1491 1493
 1494 1496 1504 1505 1510 1522 1523 1530]
Dropped 98 epochs: 9, 10, 12, 24, 27, 28, 32, 33

In [ ]:
epochs_faster.plot_drop_log()

### 1.2. Run ICA


When visualizing the components, it is recommended to subset the data (see below).


In [23]:
settings["ica"]

{'n_components': 32, 'method': 'picard', 'decim': None}

In [24]:
settings["ica"]

settings["ica"]["method"] = 'picard'
settings["ica"]["decim"] = 2

In [25]:
ica = run_ica(epochs_faster)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Loading data for 1501 events and 726 original time points ...


/Users/weian/.conda/envs/eeg-workshop/lib/python3.7/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):
/Users/weian/.conda/envs/eeg-workshop/lib/python3.7/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):


Selecting by number: 32 components
Loading data for 1501 events and 726 original time points ...
Fitting ICA took 29.4s.
EOG channels are not found. Attempting to use Fp1,Fp2 channels as EOG channels.
Using EOG channels: Fp1, Fp2
Loading data for 1501 events and 726 original time points ...
Loading data for 1501 events and 726 original time points ...
Loading data for 1501 events and 726 original time points ...
Loading data for 1501 events and 726 original time points ...


In [ ]:
# Plot component topographies
ica.plot_components()


In [ ]:
# Visualize components on epochs
# Subset epochs to reduce execution time (e.g. take epochs from every 7th event)
subset = list(epochs.event_id.keys())[::7]
# Exclude components by selecting them, right click on component name to visualize source:
ica.plot_sources(epochs_faster[subset])

In [26]:
ica.exclude

[0]

In [27]:
# After selecting the components to exclude, apply ICA to epochs
epochs_ica = apply_ica(epochs_faster, ica)

Loading data for 1501 events and 726 original time points ...
Applying ICA to Epochs instance
    Transforming to ICA space (32 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components


In [29]:
print(epochs_ica.info)

<Info | 10 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 64 EEG
 custom_ref_applied: False
 description: n_components: 1
 dig: 64 items (64 EEG)
 highpass: 0.5 Hz
 lowpass: 45.0 Hz
 meas_date: 2021-02-08 14:08:06 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 temp: 61_E_Day1_ICA
>


### 1.3. Visualize ICA cleaned epochs (optional)

This step can be repeated after each preprocessing step, or you can also do a final inspection at the end. 


In [ ]:
epochs_ica[subset].plot(n_epochs=10, n_channels=32, scalings={'eeg': 20e-6},)

### 1.4. Save cleaned epochs (recommended)

In [ ]:
epochs_path

In [ ]:
epochs_ica.info["temp"]

In [ ]:
os.path.join(epochs_path, f'{epochs_ica.info["temp"]}-epo.fif.gz')

In [ ]:
epochs_ica.save(os.path.join(epochs_path, f'{epochs_ica.info["temp"]}-epo.fif.gz'),
                overwrite=True)

### 1.5. Create a log file 

We can create a log file for the preprocessed data and store metadata
that could be useful to remember. You can add more columns to this, or 
remove the ones that are not needed. For documentation purporses, it is 
recommended to store the number of rejected and total epochs, the number of
ICA components that were rejected, the number of interpolated electrodes etc.
You can also add a column with "notes" to add custom descriptions about the data.

In [ ]:
notes = 'additional notes about the data'

In [ ]:
log_file_path = os.path.join(os.path.join(epochs_path, 'log.csv'))

In [ ]:
epochs_ica.info["temp"]

In [ ]:
update_log(log_file_path, epochs_ica, notes)

In [ ]:
log_file_path = os.path.join(os.path.join(epochs_path, 'log.csv'))
print(log_file_path)

#update_log

### 2.1. Run autoreject

In [30]:
settings['autoreject']['threshold'] = 0.15

In [31]:
reject_log = run_autoreject(epochs_ica, subset=True)

Fitting autoreject on random (n=375) subset of epochs: 
Running autoreject on ch_type=eeg


/Users/weian/.conda/envs/eeg-workshop/lib/python3.7/site-packages/mne/externals/tqdm/__init__.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(__version__) < LooseVersion('4.36'):
/Users/weian/.conda/envs/eeg-workshop/lib/python3.7/site-packages/mne/externals/tqdm/__init__.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(__version__) < LooseVersion('4.36'):


  0%|          | Creating augmented epochs : 0/64 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/64 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/375 [00:00<?,       ?it/s]

...

  0%|          | Repairing epochs : 0/375 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/375 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.60 and n_interpolate=32

AUTOREJECT report
There are 60 bad epochs found with Autoreject. You can assess these epochs with reject_log.bad_epochs

There are 205 bad epochs where more than 15% of the channels were noisy. You can assess these epochs with reject_log.report


In [ ]:
# Here you can decide how strict should be the epoch rejection.
# You can drop only those that were marked as bad epochs, or a more 
# strict rejection threshold can be if you drop epochs where more than
# 15% of the channels were marked as noisy.

# You can plot the epochs with Autoreject, where bad epochs are marked with
# red colors. 

#reject_log.plot_epochs(epochs_ica)

In [32]:
epochs_autoreject = apply_autoreject(epochs=epochs_ica, reject_log=reject_log)

Dropped 205 epochs: 2, 3, 9, 22, 23, 28, 29, 32, 33, 35, 36, 38, 39, 40, 43, 50, 62, 70, 79, 82, 99, 102, 118, 119, 123, 161, 164, 170, 173, 177, 179, 188, 189, 193, 199, 203, 205, 206, 210, 222, 223, 231, 235, 241, 242, 243, 253, 261, 265, 266, 271, 272, 279, 285, 288, 291, 292, 293, 294, 295, 297, 302, 307, 311, 320, 322, 356, 361, 369, 377, 396, 417, 437, 450, 452, 453, 464, 465, 472, 476, 479, 482, 494, 495, 502, 517, 519, 529, 555, 564, 571, 580, 594, 595, 601, 602, 608, 610, 627, 630, 634, 659, 661, 665, 678, 681, 682, 683, 685, 686, 689, 699, 703, 704, 705, 708, 710, 713, 727, 728, 732, 737, 743, 756, 758, 759, 776, 796, 802, 806, 830, 834, 835, 848, 853, 863, 878, 879, 886, 896, 905, 906, 907, 960, 976, 984, 993, 1012, 1020, 1028, 1036, 1045, 1055, 1075, 1078, 1080, 1083, 1089, 1091, 1107, 1108, 1110, 1123, 1140, 1142, 1150, 1151, 1161, 1163, 1192, 1204, 1218, 1220, 1235, 1243, 1245, 1251, 1273, 1274, 1295, 1311, 1319, 1320, 1322, 1341, 1342, 1345, 1347, 1350, 1366, 1374, 1384,

In [33]:
epochs_autoreject.info["temp"]

'61_E_Day1_ICA_autoreject'

In [ ]:
os.path.join(epochs_path, f'{epochs_autoreject.info["temp"]}-epo.fif.gz')

In [ ]:
epochs_autoreject.save(os.path.join(epochs_path, f'{epochs_autoreject.info["temp"]}-epo.fif.gz'), overwrite=True)

In [ ]:
# Update log
notes = ''

update_log(log_file_path, epochs_autoreject, notes)

### 3. Find and interpolate bad channels

In [ ]:
bads = get_noisy_channels(epochs=epochs_autoreject, with_ransac=True)

In [ ]:
#bads.extend(['T7', 'CPz'])

# .append() for string e.g. 'F7'
# .extend() for list ['F7', 'F8']

In [ ]:
bads

In [ ]:
epochs_ransac = interpolate_bad_channels(epochs=epochs_autoreject, bads=bads)

In [ ]:
print(epochs_ransac.info)

In [ ]:
# Check how many trials are left for each condition per epoch
for i in range(5):
    print(i+1, epochs[f"epoch == {i+1}& triplet_type == 'HR'"].average().nave)

In [ ]:
for i in range(5):
    print(i+1, epochs_ransac[f"epoch == {i+1}& triplet_type == 'HR'"].average().nave)

In [ ]:
epochs_ransac.metadata

In [ ]:
# inspect which sensors were interpolated (if any)
print(epochs_ransac.info)

## 4. Final visual inspection

Mark epochs that should be dropped,  etc.

In [ ]:
# # use indexing to plot fewer epochs (faster) e.g. [::7] shows only every 7th epoch
epochs_ransac[::7].plot(n_epochs=10,
                       n_channels=32,
                # group_by='position',
                       scalings={'eeg': 20e-6})

### 5.2. Set average reference

To set a “virtual reference” that is the average of all channels, you can use set_eeg_reference() with ref_channels='average'.


In [ ]:
epochs_ransac.set_eeg_reference('average')

## 6. Save cleaned epochs

In [ ]:
os.path.join(epochs_path, f'{epochs_ransac.info["temp"]}-epo.fif.gz')

In [ ]:
epochs_ransac.save(os.path.join(epochs_path, f'{epochs_ransac.info["temp"]}-epo.fif.gz'), overwrite=True)

In [ ]:
update_log(log_file_path, epochs_ransac, '')

In [ ]:
epochs.plot_sensors(show_names=True)

In [ ]:
ch_names = ['F7', 'F5', 'F3', 'FC5', 'FC3',
           'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2',
           'F4', 'F6', 'F8', 'FC4', 'FC6',
           'FT7', 'T7', 'TP7', 
           'C3', 'Cz', 'C4',
           'FT8', 'T8', 'TP8',
           'CP5', 'CP3', 'P7', 'P5', 'P3',
           'CP1', 'CPz', 'CP2', 'P1', 'Pz', 'P2',
           'CP4', 'CP6', 'P4', 'P6', 'P8',
           'PO3', 'PO7', 'O1',
           'PO4', 'PO8', 'O2',]


epochs_evoked = epochs_ransac.copy().pick_channels(ch_names, ordered=True)

In [ ]:
epochs_evoked.plot_psd()

In [ ]:
e1_H = epochs_evoked["epoch == 1 & triplet == 'H'"].average()
e1_H.apply_baseline((-0.2, 0.0))
#e1_H.comment = f"{epochs_evoked.comment}_e1_H"

e1_H.plot('F8', spatial_colors=True)